This script will run the ak and canada models together

Read in packages

In [1]:
import pandas as pd
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.python.lib.io import file_io
from tensorflow.python.keras.optimizer_v2.adam import Adam
import os
import segmentation_models as sm
import matplotlib.pyplot as plt
import numpy as np
#from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Dropout,Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.layers import concatenate, Conv2DTranspose, Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, Input, AvgPool2D
from tensorflow.keras.models import Model
from keras_unet_collection import models
import tensorflow_addons as tfa
import geopandas as gpd
import logging
# from tensorflow import tensorflow.keras.mixed_precision.set_global_policy("mixed_float16")

Segmentation Models: using `keras` framework.


/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a 

Read in the testing data for all four models, plus all the mtbs data for models 1 and 2

In [34]:
#get all the pathways
training_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_training_files.csv')['Files'].tolist()
validation_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_validation_files.csv')['Files'].tolist()
testing_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files.csv')['Files'].tolist()

model_test = testing_names

Image generator for 3 bands

In [35]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
min_max = pd.read_csv("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/l8_sent_collection2_global_min_max_cutoff_proj.csv").reset_index(drop = True)
min_max = min_max[['6', '7', '8']]

class img_gen_vi(tensorflow.keras.utils.Sequence):

    """Helper to iterate over the data (as Numpy arrays).
    Inputs are batch size, the image size, the input paths (x) and target paths (y)
    """

    #will need pre defined variables batch_size, img_size, input_img_paths and target_img_paths
    def __init__(self, batch_size, img_size, input_img_paths):
	    self.batch_size = batch_size
	    self.img_size = img_size
	    self.input_img_paths = input_img_paths
	    self.target_img_paths = input_img_paths

    #number of batches the generator is supposed to produceis the length of the paths divided by the batch siize
    def __len__(self):
	    return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_img_paths = self.input_img_paths[i : i + self.batch_size] #for a given index get the input batch pathways (x)
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size] #for a given index get the input batch pathways (y)
		
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32") #create matrix of zeros which will have the dimension height, wideth, n_bands), 8 is the n_bands
        
  
         #start populating x by enumerating over the input img paths
        for j, path in enumerate(batch_img_paths):

            #load image
            img =  np.round(np.load(path), 3)[:, :, :-1]

            # img = img * 1000
            img = img.astype(float)
            img = np.round(img, 3)
            img[img == 0] = -999

            img[np.isnan(img)] = -999


            img[img == -999] = np.nan

            in_shape = img.shape
            
            #turn to dataframe to normalize
            img = img.reshape(img.shape[0] * img.shape[1], img.shape[2])
			
            img = pd.DataFrame(img)
			
            img.columns = min_max.columns
			
            img = pd.concat([min_max, img]).reset_index(drop = True)


            #normalize 0 to 1
            img = pd.DataFrame(scaler.fit_transform(img))
			
            img = img.iloc[2:]
#
#             img = img.values.reshape(in_shape)
            img = img.values.reshape(in_shape)

#             replace nan with -1
            img[np.isnan(img)] = -1

#apply standardization
# img = normalize(img, axis=(0,1))

            img = np.round(img, 3)
            #populate x
            x[j] = img#[:, :, 4:] index number is not included, 


        #do tthe same thing for y
        y = np.zeros((self.batch_size,) + self.img_size, dtype="uint8")

        for j, path in enumerate(batch_target_img_paths):

            #load image
            img =  np.round(np.load(path), 3)[:, :, -1]

            img = img.astype(int)

            img[img < 0] = 0
            img[img >1] = 0
            img[~np.isin(img, [0,1])] = 0

            img[np.isnan(img)] = 0
            img = img.astype(int)

            # img =  tf.keras.utils.to_categorical(img, num_classes = 2)
            # y[j] = np.expand_dims(img, 2) 
            y[j] = img
  
       
    #Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
    # y[j] -= 1

        return x, y


Read in the images based on the generator

In [36]:
#batch size and img size
BATCH_SIZE = 45
GPUS = ["GPU:0", "GPU:1", "GPU:2", "GPU:3"]
strategy = tensorflow.distribute.MirroredStrategy() #can add GPUS here to select specific ones
print('Number of devices: %d' % strategy.num_replicas_in_sync) 

batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

#image size
img_size = (128, 128)

#number of classes to predict
num_classes = 1

#russia 2015 2019 testing generator
models_vi_gen =  img_gen_vi(batch_size, img_size, model_test)





Number of devices: 2


Read in all the models

In [37]:
# #nbac mtbs
# model_1 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/Russia/models/nbac_mtbs_2015_2019.tf", 
#                                            custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
#                                                            'recall':sm.metrics.Recall(threshold = 0.5),
#                                                             'f1-score': sm.metrics.FScore(threshold=0.5),
#                                                              'iou_score': sm.metrics.IOUScore(threshold=0.5),
#                                                               'accuracy': 'accuracy'})

# #russia
# model_2 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/Russia/models/russia_good_2015_2019.tf", 
#                                            custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
#                                                            'recall':sm.metrics.Recall(threshold = 0.5),
#                                                             'f1-score': sm.metrics.FScore(threshold=0.5),
#                                                              'iou_score': sm.metrics.IOUScore(threshold=0.5)})

# #combined
# model_3 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/Russia/models/combined_good_2015_2019.tf", 
#                                            custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
#                                                            'recall':sm.metrics.Recall(threshold = 0.5),
#                                                             'f1-score': sm.metrics.FScore(threshold=0.5),
#                                                              'iou_score': sm.metrics.IOUScore(threshold=0.5)})

#---------------new try
#nbac mtbs
model_1 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/Russia/models/russia_good_regularize.tf", 
                                           custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
                                                           'recall':sm.metrics.Recall(threshold = 0.5),
                                                            'f1-score': sm.metrics.FScore(threshold=0.5),
                                                             'iou_score': sm.metrics.IOUScore(threshold=0.5),
                                                              'accuracy': 'accuracy'})

# #russia
# model_2 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/Russia/models/russia_good_2015_2019_no_regularize.tf", 
#                                            custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
#                                                            'recall':sm.metrics.Recall(threshold = 0.5),
#                                                             'f1-score': sm.metrics.FScore(threshold=0.5),
#                                                              'iou_score': sm.metrics.IOUScore(threshold=0.5)})

# #combined
# model_3 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/Russia/models/combined_good_no_reg_2015_2019.tf", 
#                                            custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
#                                                            'recall':sm.metrics.Recall(threshold = 0.5),
#                                                             'f1-score': sm.metrics.FScore(threshold=0.5),
#                                                              'iou_score': sm.metrics.IOUScore(threshold=0.5)})




Model 1 testing scores

In [38]:
model_1_res = model_1.evaluate_generator(models_vi_gen, 100)

iou = model_1_res[-2]
precision = model_1_res[-5]
recall = model_1_res[-4]
f1 = model_1_res[-3]
accuracy = model_1_res[-1]

print(f"Model 1 testing IoU is {iou}")
print(f"Model 1 testing precision is {precision}")
print(f"Model 1 testing recall is {recall}")
print(f"Model 1 testing f1 is {f1}")
print(f"Model 1 testing accuracy is {accuracy}")


/tmp/ipykernel_3998823/216819093.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model_1.evaluate_generator(models_vi_gen, 100)


Model 1 testing IoU is 0.8034630417823792
Model 1 testing precision is 0.9299473762512207
Model 1 testing recall is 0.8552743792533875
Model 1 testing f1 is 0.8903196454048157
Model 1 testing accuracy is 0.9543337821960449


In [12]:
len(model_1_res)

19

Model 2 testing scores

In [9]:
model_2_res = model_2.evaluate_generator(models_vi_gen, 100)

iou = model_2_res[-2]
precision = model_2_res[-5]
recall = model_2_res[-4]
f1 = model_2_res[-3]
accuracy = model_2_res[-1]

print(f"Model 2 testing IoU is {iou}")
print(f"Model 2 testing precision is {precision}")
print(f"Model 2 testing recall is {recall}")
print(f"Model 2 testing f1 is {f1}")
print(f"Model 2 testing accuracy is {accuracy}")

/tmp/ipykernel_1343492/1093850466.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_2_res = model_2.evaluate_generator(models_vi_gen, 100)


Model 2 testing IoU is 0.5864999294281006
Model 2 testing precision is 0.7678250670433044
Model 2 testing recall is 0.7140936851501465
Model 2 testing f1 is 0.7334617376327515
Model 2 testing accuracy is 0.8893437385559082


Model 3 testing scores

In [10]:
model_3_res = model_3.evaluate_generator(models_vi_gen, 100)

iou = model_3_res[-2]
precision = model_3_res[-5]
recall = model_3_res[-4]
f1 = model_3_res[-3]
accuracy = model_3_res[-1]

print(f"Model 3 testing IoU is {iou}")
print(f"Model 3 testing precision is {precision}")
print(f"Model 3 testing recall is {recall}")
print(f"Model 3 testing f1 is {f1}")
print(f"Model 3 testing accuracy is {accuracy}")

/tmp/ipykernel_1343492/121600142.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_3_res = model_3.evaluate_generator(models_vi_gen, 100)


Model 3 testing IoU is 0.5609645247459412
Model 3 testing precision is 0.8144935369491577
Model 3 testing recall is 0.6458084583282471
Model 3 testing f1 is 0.7122849822044373
Model 3 testing accuracy is 0.8887165784835815


Model 4 testing scores

2015 Only 

In [4]:


#get all the pathways
training_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_training_files.csv')['Files'].tolist()
validation_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_validation_files.csv')['Files'].tolist()
testing_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files.csv')['Files'].tolist()

# #shapefile
in_shape = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_polygons.shp')

in_shape['Year'] = in_shape['Year'].astype(int)

remove = in_shape[in_shape['Year'] .isin ([2015])]

remove = remove[remove['ID'] .isin (['29690000000000-0000000000', '29690000000000-0000023296']) == False]
remove['ID'] = remove['ID'].astype(str) + '.npy'

remove = remove['ID'].tolist()


def filter_items_by_ending(original_list, unwanted_endings):
    """
    Filters out items from the original list that end with any of the specified unwanted endings.
    
    Parameters:
    - original_list: List of strings to be filtered.
    - unwanted_endings: List of string endings based on which items will be removed from the original list.
    
    Returns:
    - List of filtered items not ending with any of the unwanted endings.
    """
    # Use list comprehension to filter out unwanted items
    return [item for item in original_list if not any(item.endswith(ending) for ending in unwanted_endings)]

training_names = filter_items_by_ending(training_names,remove)
validation_names = filter_items_by_ending(validation_names,remove)
testing_names = filter_items_by_ending(testing_names, remove)
training_names = training_names + testing_names

#get all the pathways fro making a new testin set
training_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_training_files.csv')['Files'].tolist()
validation_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_validation_files.csv')['Files'].tolist()
testing_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files.csv')['Files'].tolist()


def keep_items_by_ending(original_list, wanted_endings):
    """
    Keeps only the items from the original list that end with any of the specified wanted endings.
    
    Parameters:
    - original_list: List of strings to be filtered.
    - wanted_endings: List of string endings based on which items will be kept in the original list.
    
    Returns:
    - List of items ending with any of the wanted endings.
    """
    # Use list comprehension to filter and keep wanted items
    return [item for item in original_list if any(item.endswith(ending) for ending in wanted_endings)]

training_names2 = keep_items_by_ending(training_names2,remove)
validation_names2 = keep_items_by_ending(validation_names2,remove)
testing_names2 = keep_items_by_ending(testing_names2, remove)

model_test = training_names2 + validation_names2 + testing_names2


In [5]:
#batch size and img size
BATCH_SIZE = 45
GPUS = ["GPU:0", "GPU:1", "GPU:2", "GPU:3"]
strategy = tensorflow.distribute.MirroredStrategy() #can add GPUS here to select specific ones
print('Number of devices: %d' % strategy.num_replicas_in_sync) 

batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

#image size
img_size = (128, 128)

#number of classes to predict
num_classes = 1

#russia 2015 2019 testing generator
models_vi_gen =  img_gen_vi(batch_size, img_size, model_test)

Number of devices: 2


In [6]:
model_1_res = model_1.evaluate_generator(models_vi_gen, 100)

iou = model_1_res[-2]
precision = model_1_res[-5]
recall = model_1_res[-4]
f1 = model_1_res[-3]
accuracy = model_1_res[-1]

print(f"Model 1 testing IoU is {iou}")
print(f"Model 1 testing precision is {precision}")
print(f"Model 1 testing recall is {recall}")
print(f"Model 1 testing f1 is {f1}")
print(f"Model 1 testing accuracy is {accuracy}")

/tmp/ipykernel_3998823/21262767.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model_1.evaluate_generator(models_vi_gen, 100)


Model 1 testing IoU is 0.5112721920013428
Model 1 testing precision is 0.8420653343200684
Model 1 testing recall is 0.5660997629165649
Model 1 testing f1 is 0.6733208298683167
Model 1 testing accuracy is 0.8786551356315613


In [8]:
't'

't'

In [9]:
model_2_res = model_2.evaluate_generator(models_vi_gen, 100)

iou = model_2_res[-2]
precision = model_2_res[-5]
recall = model_2_res[-4]
f1 = model_2_res[-3]
accuracy = model_2_res[-1]

print(f"Model 2 testing IoU is {iou}")
print(f"Model 2 testing precision is {precision}")
print(f"Model 2 testing recall is {recall}")
print(f"Model 2 testing f1 is {f1}")
print(f"Model 2 testing accuracy is {accuracy}")

/tmp/ipykernel_3998823/1093850466.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_2_res = model_2.evaluate_generator(models_vi_gen, 100)


Model 2 testing IoU is 0.5882929563522339
Model 2 testing precision is 0.7685894966125488
Model 2 testing recall is 0.7149486541748047
Model 2 testing f1 is 0.7382818460464478
Model 2 testing accuracy is 0.8881325721740723


In [10]:
model_3_res = model_3.evaluate_generator(models_vi_gen, 100)

iou = model_3_res[-2]
precision = model_3_res[-5]
recall = model_3_res[-4]
f1 = model_3_res[-3]
accuracy = model_3_res[-1]

print(f"Model 3 testing IoU is {iou}")
print(f"Model 3 testing precision is {precision}")
print(f"Model 3 testing recall is {recall}")
print(f"Model 3 testing f1 is {f1}")
print(f"Model 3 testing accuracy is {accuracy}")

/tmp/ipykernel_3998823/121600142.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_3_res = model_3.evaluate_generator(models_vi_gen, 100)


Model 3 testing IoU is 0.5640661716461182
Model 3 testing precision is 0.8235633373260498
Model 3 testing recall is 0.6427218914031982
Model 3 testing f1 is 0.7184184789657593
Model 3 testing accuracy is 0.8889491558074951


2018 only 

In [17]:


#get all the pathways
training_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_training_files.csv')['Files'].tolist()
validation_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_validation_files.csv')['Files'].tolist()
testing_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files.csv')['Files'].tolist()

# #shapefile
in_shape = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_polygons.shp')

in_shape['Year'] = in_shape['Year'].astype(int)

remove = in_shape[in_shape['Year'] .isin ([2018])]

remove = remove[remove['ID'] .isin (['29690000000000-0000000000', '29690000000000-0000023296']) == False]
remove['ID'] = remove['ID'].astype(str) + '.npy'

remove = remove['ID'].tolist()


def filter_items_by_ending(original_list, unwanted_endings):
    """
    Filters out items from the original list that end with any of the specified unwanted endings.
    
    Parameters:
    - original_list: List of strings to be filtered.
    - unwanted_endings: List of string endings based on which items will be removed from the original list.
    
    Returns:
    - List of filtered items not ending with any of the unwanted endings.
    """
    # Use list comprehension to filter out unwanted items
    return [item for item in original_list if not any(item.endswith(ending) for ending in unwanted_endings)]

training_names = filter_items_by_ending(training_names,remove)
validation_names = filter_items_by_ending(validation_names,remove)
testing_names = filter_items_by_ending(testing_names, remove)
training_names = training_names + testing_names

#get all the pathways fro making a new testin set
training_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_training_files.csv')['Files'].tolist()
validation_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_validation_files.csv')['Files'].tolist()
testing_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files.csv')['Files'].tolist()


def keep_items_by_ending(original_list, wanted_endings):
    """
    Keeps only the items from the original list that end with any of the specified wanted endings.
    
    Parameters:
    - original_list: List of strings to be filtered.
    - wanted_endings: List of string endings based on which items will be kept in the original list.
    
    Returns:
    - List of items ending with any of the wanted endings.
    """
    # Use list comprehension to filter and keep wanted items
    return [item for item in original_list if any(item.endswith(ending) for ending in wanted_endings)]

training_names2 = keep_items_by_ending(training_names2,remove)
validation_names2 = keep_items_by_ending(validation_names2,remove)
testing_names2 = keep_items_by_ending(testing_names2, remove)

model_test = training_names2 + validation_names2 + testing_names2


In [19]:
#batch size and img size
BATCH_SIZE = 45
GPUS = ["GPU:0", "GPU:1", "GPU:2", "GPU:3"]
strategy = tensorflow.distribute.MirroredStrategy() #can add GPUS here to select specific ones
print('Number of devices: %d' % strategy.num_replicas_in_sync) 

batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

#image size
img_size = (128, 128)

#number of classes to predict
num_classes = 1

#russia 2015 2019 testing generator
models_vi_gen =  img_gen_vi(batch_size, img_size, model_test)

Number of devices: 2


In [20]:
model_1_res = model_1.evaluate_generator(models_vi_gen, 100)

iou = model_1_res[-2]
precision = model_1_res[-5]
recall = model_1_res[-4]
f1 = model_1_res[-3]
accuracy = model_1_res[-1]

print(f"Model 1 testing IoU is {iou}")
print(f"Model 1 testing precision is {precision}")
print(f"Model 1 testing recall is {recall}")
print(f"Model 1 testing f1 is {f1}")
print(f"Model 1 testing accuracy is {accuracy}")

/tmp/ipykernel_3998823/21262767.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model_1.evaluate_generator(models_vi_gen, 100)


Model 1 testing IoU is 0.6879123449325562
Model 1 testing precision is 0.8904786705970764
Model 1 testing recall is 0.7526465654373169
Model 1 testing f1 is 0.8140120506286621
Model 1 testing accuracy is 0.9214476943016052


In [21]:
model_2_res = model_2.evaluate_generator(models_vi_gen, 100)

iou = model_2_res[-2]
precision = model_2_res[-5]
recall = model_2_res[-4]
f1 = model_2_res[-3]
accuracy = model_2_res[-1]

print(f"Model 2 testing IoU is {iou}")
print(f"Model 2 testing precision is {precision}")
print(f"Model 2 testing recall is {recall}")
print(f"Model 2 testing f1 is {f1}")
print(f"Model 2 testing accuracy is {accuracy}")

/tmp/ipykernel_3998823/1093850466.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_2_res = model_2.evaluate_generator(models_vi_gen, 100)


Model 2 testing IoU is 0.8963956832885742
Model 2 testing precision is 0.9531925916671753
Model 2 testing recall is 0.9369732737541199
Model 2 testing f1 is 0.9449449181556702
Model 2 testing accuracy is 0.9750679135322571


In [22]:
't'

't'

In [23]:
model_3_res = model_3.evaluate_generator(models_vi_gen, 100)

iou = model_3_res[-2]
precision = model_3_res[-5]
recall = model_3_res[-4]
f1 = model_3_res[-3]
accuracy = model_3_res[-1]

print(f"Model 3 testing IoU is {iou}")
print(f"Model 3 testing precision is {precision}")
print(f"Model 3 testing recall is {recall}")
print(f"Model 3 testing f1 is {f1}")
print(f"Model 3 testing accuracy is {accuracy}")

/tmp/ipykernel_3998823/121600142.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_3_res = model_3.evaluate_generator(models_vi_gen, 100)


Model 3 testing IoU is 0.788118839263916
Model 3 testing precision is 0.9034174084663391
Model 3 testing recall is 0.8606230616569519
Model 3 testing f1 is 0.8807426691055298
Model 3 testing accuracy is 0.9467858672142029


In [24]:
't'

't'

2010 only 

In [28]:
in_shape = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_polygons.shp')

in_shape['Year'] = in_shape['Year'].astype(int)

remove = in_shape[in_shape['Year'] .isin ([2010])]

emove = remove[remove['ID'] .isin (['29690000000000-0000000000', '29690000000000-0000023296']) == False]
remove['ID'] = remove['ID'].astype(str) + '.npy'

remove = remove['ID'].tolist()


def filter_items_by_ending(original_list, unwanted_endings):
    """
    Filters out items from the original list that end with any of the specified unwanted endings.
    
    Parameters:
    - original_list: List of strings to be filtered.
    - unwanted_endings: List of string endings based on which items will be removed from the original list.
    
    Returns:
    - List of filtered items not ending with any of the unwanted endings.
    """
    # Use list comprehension to filter out unwanted items
    return [item for item in original_list if not any(item.endswith(ending) for ending in unwanted_endings)]

training_names = filter_items_by_ending(training_names,remove)
validation_names = filter_items_by_ending(validation_names,remove)
testing_names = filter_items_by_ending(testing_names, remove)
training_names = training_names + testing_names

#get all the pathways fro making a new testin set
training_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_training_files.csv')['Files'].tolist()
validation_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_validation_files.csv')['Files'].tolist()
testing_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files.csv')['Files'].tolist()


def keep_items_by_ending(original_list, wanted_endings):
    """
    Keeps only the items from the original list that end with any of the specified wanted endings.
    
    Parameters:
    - original_list: List of strings to be filtered.
    - wanted_endings: List of string endings based on which items will be kept in the original list.
    
    Returns:
    - List of items ending with any of the wanted endings.
    """
    # Use list comprehension to filter and keep wanted items
    return [item for item in original_list if any(item.endswith(ending) for ending in wanted_endings)]

training_names2 = keep_items_by_ending(training_names2,remove)
validation_names2 = keep_items_by_ending(validation_names2,remove)
testing_names2 = keep_items_by_ending(testing_names2, remove)

model_test = training_names2 + validation_names2 + testing_names2



/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [29]:
#batch size and img size
BATCH_SIZE = 45
GPUS = ["GPU:0", "GPU:1", "GPU:2", "GPU:3"]
strategy = tensorflow.distribute.MirroredStrategy() #can add GPUS here to select specific ones
print('Number of devices: %d' % strategy.num_replicas_in_sync) 

batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

#image size
img_size = (128, 128)

#number of classes to predict
num_classes = 1

#russia 2015 2019 testing generator
models_vi_gen =  img_gen_vi(batch_size, img_size, model_test)

Number of devices: 2


In [30]:
model_3_res = model_3.evaluate_generator(models_vi_gen, 100)

iou = model_3_res[-2]
precision = model_3_res[-5]
recall = model_3_res[-4]
f1 = model_3_res[-3]
accuracy = model_3_res[-1]

print(f"Model 3 testing IoU is {iou}")
print(f"Model 3 testing precision is {precision}")
print(f"Model 3 testing recall is {recall}")
print(f"Model 3 testing f1 is {f1}")
print(f"Model 3 testing accuracy is {accuracy}")

/tmp/ipykernel_3998823/121600142.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_3_res = model_3.evaluate_generator(models_vi_gen, 100)


Model 3 testing IoU is 0.825490415096283
Model 3 testing precision is 0.959470272064209
Model 3 testing recall is 0.8551809191703796
Model 3 testing f1 is 0.9037761092185974
Model 3 testing accuracy is 0.9658539295196533


2011 Only 

In [31]:
in_shape = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_polygons.shp')

in_shape['Year'] = in_shape['Year'].astype(int)

remove = in_shape[in_shape['Year'] .isin ([2011])]

emove = remove[remove['ID'] .isin (['29690000000000-0000000000', '29690000000000-0000023296']) == False]
remove['ID'] = remove['ID'].astype(str) + '.npy'

remove = remove['ID'].tolist()


def filter_items_by_ending(original_list, unwanted_endings):
    """
    Filters out items from the original list that end with any of the specified unwanted endings.
    
    Parameters:
    - original_list: List of strings to be filtered.
    - unwanted_endings: List of string endings based on which items will be removed from the original list.
    
    Returns:
    - List of filtered items not ending with any of the unwanted endings.
    """
    # Use list comprehension to filter out unwanted items
    return [item for item in original_list if not any(item.endswith(ending) for ending in unwanted_endings)]

training_names = filter_items_by_ending(training_names,remove)
validation_names = filter_items_by_ending(validation_names,remove)
testing_names = filter_items_by_ending(testing_names, remove)
training_names = training_names + testing_names

#get all the pathways fro making a new testin set
training_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_training_files.csv')['Files'].tolist()
validation_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_validation_files.csv')['Files'].tolist()
testing_names2 = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files.csv')['Files'].tolist()


def keep_items_by_ending(original_list, wanted_endings):
    """
    Keeps only the items from the original list that end with any of the specified wanted endings.
    
    Parameters:
    - original_list: List of strings to be filtered.
    - wanted_endings: List of string endings based on which items will be kept in the original list.
    
    Returns:
    - List of items ending with any of the wanted endings.
    """
    # Use list comprehension to filter and keep wanted items
    return [item for item in original_list if any(item.endswith(ending) for ending in wanted_endings)]

training_names2 = keep_items_by_ending(training_names2,remove)
validation_names2 = keep_items_by_ending(validation_names2,remove)
testing_names2 = keep_items_by_ending(testing_names2, remove)

model_test = training_names2 + validation_names2 + testing_names2



/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [32]:
#batch size and img size
BATCH_SIZE = 45
GPUS = ["GPU:0", "GPU:1", "GPU:2", "GPU:3"]
strategy = tensorflow.distribute.MirroredStrategy() #can add GPUS here to select specific ones
print('Number of devices: %d' % strategy.num_replicas_in_sync) 

batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

#image size
img_size = (128, 128)

#number of classes to predict
num_classes = 1

#russia 2015 2019 testing generator
models_vi_gen =  img_gen_vi(batch_size, img_size, model_test)

Number of devices: 2


In [33]:
model_3_res = model_3.evaluate_generator(models_vi_gen, 100)

iou = model_3_res[-2]
precision = model_3_res[-5]
recall = model_3_res[-4]
f1 = model_3_res[-3]
accuracy = model_3_res[-1]

print(f"Model 3 testing IoU is {iou}")
print(f"Model 3 testing precision is {precision}")
print(f"Model 3 testing recall is {recall}")
print(f"Model 3 testing f1 is {f1}")
print(f"Model 3 testing accuracy is {accuracy}")

/tmp/ipykernel_3998823/121600142.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_3_res = model_3.evaluate_generator(models_vi_gen, 100)


Model 3 testing IoU is 0.8015790581703186
Model 3 testing precision is 0.9532454609870911
Model 3 testing recall is 0.8342726826667786
Model 3 testing f1 is 0.8891804218292236
Model 3 testing accuracy is 0.9497191905975342
